# Optimizing Panda Dataframes in chunks and exploratory data engineering

In this exercise, we'll be analyzing the data footprint of a loans dataset from the Lending Club from 2007 and transforming the data to optimize it for memory usage. 

In [130]:
import pandas as pd
pd.options.display.max_columns = 99

In [131]:
# Import the header and first five lines of data to look for
# any potential data quality issues

loans_data_check = pd.read_csv("loans_2007.csv", nrows=6)

loans_data_check.head(6)



,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
5,1075269,1311441.0,5000.0,5000.0,5000.0,36 months,7.90%,156.46,A,A4,Veolia Transportaton,3 years,RENT,36000.0,Source Verified,Dec-2011,Fully Paid,n,wedding,My wedding loan I promise to pay back,852xx,AZ,11.20,0.0,Nov-2004,3.0,9.0,0.0,7963.0,28.3%,12.0,f,0.00,0.00,5632.210000,5632.21,5000.00,632.21,0.00,0.00,0.00,Jan-2015,161.03,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [132]:
# Read in the first 1000 rows and calculate memory usage.
# We'll increase or decrease accordingly to stay under 5 MB.

data = pd.read_csv("loans_2007.csv", nrows=1000)

total_mem = data.memory_usage(deep=True)

total_mem_sum = total_mem.sum()

total_mem_used = total_mem_sum / (1024*1024)

print(total_mem_used)

1.5502090454101562


In [133]:
# It looks like 3000 rows uses just under 5MB of memory per chunk.

data = pd.read_csv("loans_2007.csv", chunksize=3000)

for chunk in data:
    print(chunk.memory_usage(deep=True).sum() / (1024*1024))

4.649013519287109
4.6447601318359375
4.646517753601074
4.647870063781738
4.6440629959106445
4.6459455490112305
4.644536972045898
4.646905899047852
4.645031929016113
4.645082473754883
4.657794952392578
4.6566619873046875
4.663469314575195
4.896910667419434
0.8808088302612305


In [134]:
data = pd.read_csv("loans_2007.csv", chunksize=3000)

total_rows = 0

for chunk in data:
    total_rows += len(chunk)
    
print(total_rows)

42538


In [135]:
# import numpy and chunk out the data set so we can see the
# column types in chunks.
import numpy as np

data_chunk = pd.read_csv("loans_2007.csv", chunksize=3000)
numeric_cols = []
string_cols = []

for chunk in data_chunk:
    numeric_cols.append(chunk.select_dtypes(include=[np.number]).shape[1])
    string_cols.append(chunk.select_dtypes(include=["object"]).shape[1])

print(numeric_cols)
print(string_cols)

[31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 30, 30]
[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22]


In [136]:
# # What are the string columns across chunks?
string_cols = []
data = pd.read_csv("loans_2007.csv", chunksize=3000)

for chunk in data:
    chunk_string_cols = chunk.select_dtypes(include=["object"]).columns.tolist()
    if len(string_cols) > 0:
        same_type = string_cols == chunk_string_cols
        if not same_type:
            print("overall string cols:", string_cols, "\n")
            print("chunk string cols:", chunk_string_cols, "\n")
    else:
        string_cols = chunk_string_cols

overall string cols: ['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type'] 

chunk string cols: ['id', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type'] 

overall string cols: ['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_

## Preliminary Data Structure Findings ##

* It looks like there are approximately 31 numeric columns and 20 string columns
* It looks like the "id" field is being counted as a string in some of our chunked data. We can disregard this as it's not a useful field for any analysis.

## How many unique values are there in string columns? How many are less than 50% unique? ##

In [137]:
data = pd.read_csv("loans_2007.csv", chunksize=3000)

unique_string_values = {}

for chunk in data:
    only_strings = chunk.select_dtypes(include=["object"])
    columns = only_strings.columns
    for column in columns:
        val_counts = only_strings[column].value_counts()
        if column in unique_string_values:
            unique_string_values[column].append(val_counts)
        else:
            unique_string_values[column] = [val_counts]
            

unique_strings_combined = {}
unique_string_stats = {
    'column_name': [],
    'total_values': [],
    'unique_values': [],
}

for column in unique_string_values:
    unique_concat = pd.concat(unique_string_values[column])
    unique_group = unique_concat.groupby(unique_concat.index).sum()
    unique_strings_combined[column] = unique_group
    if unique_group.shape[0] < 50:
        print(column, unique_group.shape[0])


initial_list_status 1
term 2
pymnt_plan 2
verification_status 3
purpose 14
home_ownership 5
application_type 1
sub_grade 35
grade 7
loan_status 9
emp_length 11


## What float columns have no missing values and could be converted to integers? ##

In [138]:
data = pd.read_csv('loans_2007.csv',chunksize=3000)

missing_values = []
for chunk in data:
    floats = chunk.select_dtypes(include=["float"])
    missing_values.append(floats.apply(pd.isnull).sum())

combined_missing_values = pd.concat(missing_values)

print(combined_missing_values.groupby(combined_missing_values.index).sum().sort_values())

member_id                        3
total_rec_int                    3
total_pymnt_inv                  3
total_pymnt                      3
revol_bal                        3
recoveries                       3
policy_code                      3
out_prncp_inv                    3
out_prncp                        3
total_rec_late_fee               3
loan_amnt                        3
last_pymnt_amnt                  3
total_rec_prncp                  3
funded_amnt_inv                  3
funded_amnt                      3
dti                              3
collection_recovery_fee          3
installment                      3
annual_inc                       7
inq_last_6mths                  32
total_acc                       32
delinq_2yrs                     32
pub_rec                         32
delinq_amnt                     32
open_acc                        32
acc_now_delinq                  32
tax_liens                      108
collections_12_mths_ex_med     148
chargeoff_within_12_

## Total Memory Usage Across Chunks ## 

In [139]:
data = pd.read_csv("loans_2007.csv", chunksize=3000)

total_memory_usage = []

for chunk in data:
    total_memory_usage.append(chunk.memory_usage(deep=True).sum() / 1024 ** 2)

print(sum(total_memory_usage))

66.2153730392456


## Data Conversion ## 

* To maximize memory usage, we convert all columns that are less than 50% unique to a category BOOLEAN type
* All numeric columns will be converted to float data types

In [140]:
string_cols

['term',
 'int_rate',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'earliest_cr_line',
 'revol_util',
 'initial_list_status',
 'last_pymnt_d',
 'last_credit_pull_d',
 'application_type']

In [141]:
useful_string_columns = ['term', 'sub_grade', 'emp_title',
                         'home_ownership', 'verification_status',
                         'issue_d', 'purpose', 'earliest_cr_line',
                         'revol_util', 'last_pymnt_d',
                         'last_credit_pull_d']




In [142]:
data = pd.read_csv("loans_2007.csv", chunksize=3000)

chunk.head(5)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
42000,247286,247257.0,6000.0,6000.0,4201.940000,36 months,11.34%,197.40,C,C2,Best Buy,3 years,RENT,32000.0,Not Verified,Feb-2008,Does not meet the credit policy. Status:Fully ...,n,debt_consolidation,Paying off Bills,326xx,FL,21.90,0.0,Jul-2000,0.0,5.0,0.0,7152.0,100.7%,12.0,f,0.0,0.0,7106.392519,4960.00,6000.00,1106.40,0.0,0.00,0.00,Feb-2011,207.82,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,NaN,0.0
42001,246996,244258.0,17250.0,17250.0,12150.005316,36 months,17.66%,620.70,G,G2,CVS PHARMACY,2 years,OWN,62000.0,Not Verified,Feb-2008,Does not meet the credit policy. Status:Charge...,n,debt_consolidation,School expenses from residency,787xx,TX,20.44,0.0,Mar-1989,7.0,16.0,0.0,28776.0,51.9%,19.0,f,0.0,0.0,4344.890000,3058.87,2278.94,2065.95,0.0,0.00,0.00,Nov-2008,621.01,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,NaN,0.0
42002,246720,246706.0,13000.0,13000.0,7700.000000,36 months,15.13%,451.48,E,E4,General Motors,7 years,RENT,78000.0,Not Verified,Feb-2008,Does not meet the credit policy. Status:Fully ...,n,debt_consolidation,Credit Card Consolidation,480xx,MI,29.28,0.0,Dec-1998,3.0,6.0,0.0,61057.0,80.7%,17.0,f,0.0,0.0,16253.158419,9626.87,13000.00,3253.16,0.0,0.00,0.00,Feb-2011,470.66,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,NaN,0.0
42003,246535,246427.0,12000.0,12000.0,5650.000000,36 months,18.29%,435.58,G,G4,usa medical center,6 years,RENT,62000.0,Not Verified,Feb-2008,Does not meet the credit policy. Status:Fully ...,n,debt_consolidation,want to have one bill instead of 12,366xx,AL,25.26,0.0,Jul-1995,3.0,14.0,0.0,13560.0,57.2%,41.0,f,0.0,0.0,15680.712874,7383.00,12000.00,3680.71,0.0,0.00,0.00,Feb-2011,450.84,Jun-2011,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,NaN,0.0
42004,246197,217842.0,4000.0,4000.0,3849.997562,36 months,10.08%,129.22,B,B3,InvestSource Inc,4 years,RENT,48000.0,Not Verified,Feb-2008,Does not meet the credit policy. Status:Charge...,n,debt_consolidation,Need to pay credit card debt,928xx,CA,0.75,0.0,Sep-2005,0.0,4.0,0.0,3403.0,74%,4.0,f,0.0,0.0,3281.400000,3158.71,2637.73,582.00,0.0,61.67,0.85,Mar-2010,129.22,Aug-2010,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,NaN,0.0


In [143]:
date_columns = ["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"]

chunk[date_columns]

,issue_d,earliest_cr_line,last_pymnt_d,last_credit_pull_d
42000,Feb-2008,Jul-2000,Feb-2011,Jun-2016
42001,Feb-2008,Mar-1989,Nov-2008,Jun-2016
42002,Feb-2008,Dec-1998,Feb-2011,Jun-2016
42003,Feb-2008,Jul-1995,Feb-2011,Jun-2011
42004,Feb-2008,Sep-2005,Mar-2010,Aug-2010
42005,Feb-2008,Nov-2000,Mar-2010,Apr-2010
42006,Feb-2008,Dec-1995,Apr-2010,Jun-2016
42007,Feb-2008,Feb-2000,Feb-2009,Oct-2009
42008,Feb-2008,Oct-1992,Jan-2010,Jun-2016
42009,Feb-2008,Oct-1994,Jul-2008,Jun-2016


In [144]:
data = pd.read_csv("loans_2007.csv", chunksize=3000)

chunk[useful_string_columns]

,term,sub_grade,emp_title,home_ownership,verification_status,issue_d,purpose,earliest_cr_line,revol_util,last_pymnt_d,last_credit_pull_d
42000,36 months,C2,Best Buy,RENT,Not Verified,Feb-2008,debt_consolidation,Jul-2000,100.7%,Feb-2011,Jun-2016
42001,36 months,G2,CVS PHARMACY,OWN,Not Verified,Feb-2008,debt_consolidation,Mar-1989,51.9%,Nov-2008,Jun-2016
42002,36 months,E4,General Motors,RENT,Not Verified,Feb-2008,debt_consolidation,Dec-1998,80.7%,Feb-2011,Jun-2016
42003,36 months,G4,usa medical center,RENT,Not Verified,Feb-2008,debt_consolidation,Jul-1995,57.2%,Feb-2011,Jun-2011
42004,36 months,B3,InvestSource Inc,RENT,Not Verified,Feb-2008,debt_consolidation,Sep-2005,74%,Mar-2010,Aug-2010
42005,36 months,D1,kaiser,MORTGAGE,Not Verified,Feb-2008,debt_consolidation,Nov-2000,37.6%,Mar-2010,Apr-2010
42006,36 months,C1,Panoramic Software,MORTGAGE,Not Verified,Feb-2008,debt_consolidation,Dec-1995,56%,Apr-2010,Jun-2016
42007,36 months,D5,AT&T Inc.,RENT,Not Verified,Feb-2008,small_business,Feb-2000,49.7%,Feb-2009,Oct-2009
42008,36 months,D4,D&Y,RENT,Not Verified,Feb-2008,debt_consolidation,Oct-1992,71.5%,Jan-2010,Jun-2016
42009,36 months,D5,Frederick's of Hollywood,RENT,Not Verified,Feb-2008,debt_consolidation,Oct-1994,49%,Jul-2008,Jun-2016


In [163]:
convert_column_dtypes = {
    "sub_grade": "category", "home_ownership": "category", 
    "verification_status": "category", "purpose": "category"
}
data = pd.read_csv('loans_2007.csv', chunksize=3000, dtype=convert_column_dtypes, parse_dates=date_columns)

missing_values_counts = {}
for chunk in data:
    cleaned_term = chunk['term'].str.lstrip(" ").str.rstrip(" months")
    revol_cleaned_no_percent = chunk['revol_util'].str.rstrip("%")
    chunk['term'] = pd.to_numeric(cleaned_term)
    chunk['revol_util'] = pd.to_numeric(revol_cleaned_no_percent)
    chunk = chunk.dropna(how='all')
    float_cols = chunk.select_dtypes(include=['float'])
    for col in float_cols.columns:
        missing_values = len(chunk) - chunk[col].count()
        if col in missing_values_counts:
            missing_values_counts[col] = missing_values_counts[col] + missing_values
        else:
            missing_values_counts[col] = missing_values
print(chunk.dtypes)
print(missing_values_counts)



id                                    object
member_id                            float64
loan_amnt                            float64
funded_amnt                          float64
funded_amnt_inv                      float64
term                                 float64
int_rate                              object
installment                          float64
grade                                 object
sub_grade                           category
emp_title                             object
emp_length                            object
home_ownership                      category
annual_inc                           float64
verification_status                 category
issue_d                       datetime64[ns]
loan_status                           object
pymnt_plan                            object
purpose                             category
title                                 object
zip_code                              object
addr_state                            object
dti       

In [165]:
total_mem = chunk.memory_usage(deep=True)

total_mem_sum = total_mem.sum()

total_mem_used = total_mem_sum / (1024*1024)

print(total_mem_used)

0.5862674713134766


## Results ## 

* Converted data had a significant impact, reducing our data footprint from ~60 MB to 